<a href="https://colab.research.google.com/github/mishkajethwani/summer-research-24/blob/main/Actual-Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Important Installations
! pip install --quiet langchain
! pip install -qU --quiet langchain-openai
! pip install --quiet langchain_community
! pip install --quiet -qU langchain-openai
! pip install --upgrade --quiet langchain langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.8 MB/s eta 0:00:00


In [2]:
#Setting the API key
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

··········


In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# Function to get History

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [45]:
from langchain_core.tools import tool

@tool
def subtract(integer_1: int, integer_2: int) -> int:
    """Subtract integer_2 from integer_1."""
    return integer_1 - integer_2

@tool
def multiply(integer_1: int, integer_2: int) -> int:
    """Multiply two integers."""
    return integer_1 * integer_2

@tool
def divide(integer_1: float, integer_2: float) -> float:
    """Divide integer_1 by integer_2. Returns a float."""
    return integer_1 / integer_2

@tool
def modulo(integer_1: int, integer_2: int) -> int:
    """Calculate the remainder of division."""
    return integer_1 % integer_2


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def factorial(integer_1: int) -> int:
    """Calculate the factorial of a non-negative integer."""
    result = 1
    for i in range(2, integer_1 + 1):
        result *= i
    return result

tools = [subtract, multiply, divide, modulo, factorial, add, exponentiate]

def setup_math_agent():
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a precise mathematical calculator. Follow these steps for each calculation:
        1. Break down the problem into individual operations.
        2. Perform each operation step-by-step, using the appropriate tool for each.
        3. Show your work clearly, stating which tool you're using and the numbers you're inputting.
        4. Double-check your work and provide the final answer.
        Available tools: subtract, multiply, divide, modulo, factorial, add, exponentiate"""),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
from langchain_core.messages import HumanMessage, AIMessage

def chat_bot_normal():
    session_id = "abc12"
    config = {"configurable": {"session_id": session_id}}

    print("Chatbot: Hello! How can I assist you today?")

    while True:
        user_input = input("You: ")
        messages = [HumanMessage(content=user_input)]
        response = with_message_history.invoke(messages, config)
        bot_response = response.content


        print("Chatbot:", bot_response)


In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

tools = [subtract, multiply, divide, modulo, factorial, add, exponentiate]

def setup_math_agent():
    llm = ChatOpenAI(temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a precise mathematical calculator. Follow these steps for each calculation:
        1. Break down the problem into individual operations.
        2. Perform each operation step-by-step, using the appropriate tool for each.
        3. Show your work clearly, stating which tool you're using and the numbers you're inputting.
        4. Double-check your work and provide the final answer.
        Available tools: subtract, multiply, divide, modulo, factorial, add, exponentiate"""),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

def chatbot_with_calculator():
    llm = ChatOpenAI(temperature=0.7)
    math_agent = setup_math_agent()

    math_tool = Tool(
        name="math_calculator",
        func=lambda x: math_agent.invoke({"input": x})['output'],
        description="Performs complex mathematical calculations. Input should be a mathematical problem."
    )
    chatbot_tools = [math_tool]
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chatbot_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant capable of general conversation and mathematical calculations.
        For mathematical problems, use the math_calculator tool. For all other queries, respond normally.
        Always maintain context from previous messages and use that information in your responses.
        If you're not sure about something, admit that you don't know or don't have that information."""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    chatbot_agent = create_openai_tools_agent(llm, chatbot_tools, chatbot_prompt)
    chatbot_executor = AgentExecutor(agent=chatbot_agent, tools=chatbot_tools, memory=memory, verbose=True)

    print("Chatbot: Hello! I'm a chatbot that can also do mathematical calculations. How can I assist you today?")

    while True:
        user_input = input("You: ")
        result = chatbot_executor.invoke({"input": user_input})
        bot_response = result['output']

chatbot_with_calculator()

Chatbot: Hello! I'm a chatbot that can handle conversations and do mathematical calculations. How can I assist you today?
You: sum of 2 plus thirty five 


> Entering new AgentExecutor chain...

Invoking: `math_calculator` with `2 + 35`




> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': 2, 'second_int': 35}`


37The sum of 2 and 35 is 37.

> Finished chain.
The sum of 2 and 35 is 37.The sum of 2 and 35 is 37.

> Finished chain.


KeyboardInterrupt: Interrupted by user

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant skilled in mathematical calculations. The user may not always require tools and may just want to chat. If the user does require a calculation:

    For each step of the calculation:
    1. Clearly state what operation you're performing.
    2. Show the exact numbers you're using, ensuring you use the correct results from previous steps.
    3. Use the appropriate tool to perform the calculation.
    4. State the result of each step.

    After completing all steps, provide the final answer. Double-check your work to ensure accuracy."""),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result and then finally subtract 2"
    }
)




> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'integer_1': 243, 'integer_2': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025
Invoking: `subtract` with `{'integer_1': 13286025, 'integer_2': 2}`


13286023The final result after following the steps you provided is 13,286,023.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result and then finally subtract 2',
 'output': 'The final result after following the steps you provided is 13,286,023.'}